In [1]:
model_checkpoint = "google/vit-base-patch16-224-in21k"  # pre-trained model from which to fine-tune

In [2]:
from datasets import load_dataset

In [3]:
data = load_dataset('imagefolder', data_dir="new")

Resolving data files:   0%|          | 0/12719 [00:00<?, ?it/s]

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 12719
    })
})

In [5]:
data = data['train'].train_test_split(test_size = 0.1, shuffle = True)
train_ds = data['train']
val_ds = data['test']

In [6]:
train_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 11447
})

In [7]:
val_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 1272
})

In [8]:
test_ds = load_dataset('imagefolder', data_dir='Benchmark/')['train']

Resolving data files:   0%|          | 0/600 [00:00<?, ?it/s]

In [9]:
train_ds[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=L size=182x182>,
 'label': 1}

In [10]:
labels = data['train'].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

label2id

{'Mild_Demented': '0',
 'Moderate_Demented': '1',
 'Non_Demented': '2',
 'Very_Mild_Demented': '3'}

In [11]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [12]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [13]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

In [14]:
train_ds = train_ds.with_transform(transforms)
val_ds = val_ds.with_transform(transforms)
test_ds = test_ds.with_transform(transforms)

In [15]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [16]:
import evaluate

accuracy = evaluate.load("accuracy")

In [17]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [18]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
training_args = TrainingArguments(output_dir="alzheimer-image-classification-google-vit-base-patch16",
                                  remove_unused_columns=False,
                                  evaluation_strategy="epoch",
                                  overwrite_output_dir=True,
                                  save_strategy="epoch",
                                  learning_rate=5e-5,
                                  per_device_train_batch_size=4,
                                  gradient_accumulation_steps=4,
                                  per_device_eval_batch_size=4,
                                  num_train_epochs=10,
                                  warmup_ratio=0.1,
                                  logging_steps=10,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="accuracy",
                                  report_to='tensorboard',
                                  push_to_hub=True)

trainer = Trainer(model=model,
                  args=training_args,
                  data_collator=data_collator,
                  train_dataset=train_ds,
                  eval_dataset=val_ds,
                  tokenizer=image_processor,
                  compute_metrics=compute_metrics)

trainer.train()

Cloning https://huggingface.co/AhmadHakami/alzheimer-image-classification-google-vit-base-patch16 into local empty directory.
c:\Users\User\anaconda3\envs\sic2\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/7150 [00:00<?, ?it/s]

{'loss': 1.403, 'learning_rate': 6.993006993006994e-07, 'epoch': 0.01}
{'loss': 1.3965, 'learning_rate': 1.3986013986013987e-06, 'epoch': 0.03}
{'loss': 1.396, 'learning_rate': 2.0979020979020983e-06, 'epoch': 0.04}
{'loss': 1.3901, 'learning_rate': 2.7972027972027974e-06, 'epoch': 0.06}
{'loss': 1.3696, 'learning_rate': 3.496503496503497e-06, 'epoch': 0.07}
{'loss': 1.3617, 'learning_rate': 4.195804195804197e-06, 'epoch': 0.08}
{'loss': 1.3571, 'learning_rate': 4.895104895104895e-06, 'epoch': 0.1}
{'loss': 1.3373, 'learning_rate': 5.594405594405595e-06, 'epoch': 0.11}
{'loss': 1.3147, 'learning_rate': 6.2937062937062944e-06, 'epoch': 0.13}
{'loss': 1.2924, 'learning_rate': 6.993006993006994e-06, 'epoch': 0.14}
{'loss': 1.2683, 'learning_rate': 7.692307692307694e-06, 'epoch': 0.15}
{'loss': 1.2507, 'learning_rate': 8.391608391608393e-06, 'epoch': 0.17}
{'loss': 1.2024, 'learning_rate': 9.090909090909091e-06, 'epoch': 0.18}
{'loss': 1.1831, 'learning_rate': 9.79020979020979e-06, 'epoch'

  0%|          | 0/318 [00:00<?, ?it/s]

{'eval_loss': 0.7519570589065552, 'eval_accuracy': 0.64937106918239, 'eval_runtime': 11.4049, 'eval_samples_per_second': 111.531, 'eval_steps_per_second': 27.883, 'epoch': 1.0}
{'loss': 0.8059, 'learning_rate': 4.996114996114996e-05, 'epoch': 1.01}
{'loss': 0.6811, 'learning_rate': 4.988344988344989e-05, 'epoch': 1.02}
{'loss': 0.6353, 'learning_rate': 4.9805749805749805e-05, 'epoch': 1.03}
{'loss': 0.6931, 'learning_rate': 4.972804972804973e-05, 'epoch': 1.05}
{'loss': 0.711, 'learning_rate': 4.9650349650349656e-05, 'epoch': 1.06}
{'loss': 0.6704, 'learning_rate': 4.9572649572649575e-05, 'epoch': 1.08}
{'loss': 0.6915, 'learning_rate': 4.94949494949495e-05, 'epoch': 1.09}
{'loss': 0.6958, 'learning_rate': 4.941724941724942e-05, 'epoch': 1.1}
{'loss': 0.682, 'learning_rate': 4.9339549339549344e-05, 'epoch': 1.12}
{'loss': 0.6294, 'learning_rate': 4.926184926184926e-05, 'epoch': 1.13}
{'loss': 0.6122, 'learning_rate': 4.918414918414919e-05, 'epoch': 1.15}
{'loss': 0.6943, 'learning_rate

...

  0%|          | 0/318 [00:00<?, ?it/s]

{'eval_loss': 0.23014533519744873, 'eval_accuracy': 0.9158805031446541, 'eval_runtime': 9.0361, 'eval_samples_per_second': 140.768, 'eval_steps_per_second': 35.192, 'epoch': 9.0}
{'loss': 0.1255, 'learning_rate': 5.516705516705517e-06, 'epoch': 9.0}
{'loss': 0.1976, 'learning_rate': 5.4390054390054395e-06, 'epoch': 9.01}
{'loss': 0.115, 'learning_rate': 5.361305361305361e-06, 'epoch': 9.03}
{'loss': 0.1257, 'learning_rate': 5.283605283605284e-06, 'epoch': 9.04}
{'loss': 0.1573, 'learning_rate': 5.205905205905206e-06, 'epoch': 9.06}
{'loss': 0.1423, 'learning_rate': 5.128205128205128e-06, 'epoch': 9.07}
{'loss': 0.0864, 'learning_rate': 5.050505050505051e-06, 'epoch': 9.08}
{'loss': 0.0992, 'learning_rate': 4.972804972804973e-06, 'epoch': 9.1}
{'loss': 0.1676, 'learning_rate': 4.895104895104895e-06, 'epoch': 9.11}
{'loss': 0.1576, 'learning_rate': 4.817404817404818e-06, 'epoch': 9.13}
{'loss': 0.241, 'learning_rate': 4.73970473970474e-06, 'epoch': 9.14}
{'loss': 0.162, 'learning_rate': 

  0%|          | 0/318 [00:00<?, ?it/s]

{'eval_loss': 0.21270409226417542, 'eval_accuracy': 0.9261006289308176, 'eval_runtime': 5.5086, 'eval_samples_per_second': 230.91, 'eval_steps_per_second': 57.728, 'epoch': 9.99}
{'train_runtime': 2384.637, 'train_samples_per_second': 48.003, 'train_steps_per_second': 2.998, 'train_loss': 0.394019101883148, 'epoch': 9.99}


TrainOutput(global_step=7150, training_loss=0.394019101883148, metrics={'train_runtime': 2384.637, 'train_samples_per_second': 48.003, 'train_steps_per_second': 2.998, 'train_loss': 0.394019101883148, 'epoch': 9.99})

In [20]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/327M [00:00<?, ?B/s]

Upload file runs/Aug14_17-07-38_AhmadHakami/events.out.tfevents.1692022063.AhmadHakami.5056.0:   0%|          …

To https://huggingface.co/AhmadHakami/alzheimer-image-classification-google-vit-base-patch16
   1ce4208..b51bc55  main -> main

To https://huggingface.co/AhmadHakami/alzheimer-image-classification-google-vit-base-patch16
   b51bc55..fac7c42  main -> main



'https://huggingface.co/AhmadHakami/alzheimer-image-classification-google-vit-base-patch16/commit/b51bc553c28d79e0a286f4ce5938893000591000'